In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'png'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L2x1c3RyZS9jb2xsaW5mL21pYXNocy0yLWFkdmFuY2VkLXByb2dyYW1taW5nLXBhcmFsbGVsLWNvbXB1dGluZy0yMDI0LTIwMjUvQ291cnNlcy9Tb2x1dGlvbnM='
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
# globals()["__spec__"] = None

{"/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/importlib/_bootstrap.py": 1723143691.6031358, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/importlib/_bootstrap_external.py": 1723143691.6031358, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/zipimport.py": 1723143690.9391248, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/codecs.py": 1723143690.64312, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/encodings/aliases.py": 1723143691.1591284, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/encodings/__init__.py": 1723143691.25913, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/encodings/utf_8.py": 1723143691.367132, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/abc.py": 1723143690.6151195, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/io.py": 1723143690.7311215, "/nfs/home/collinf/micromamba/envs/miashs-hpc/lib/python3.12/stat.py": 1723143690.8671236, "/nfs/home/

In [2]:
## Example from PEP 0255
def fib():
    a, b = 0, 1
    while 1:
        yield b
        a, b = b, a + b

In [3]:
gen1 = fib()

# prints the first 10 fibonnaci numbers
for i in range(10):
    print(next(gen1), end=', ')
print("\nPassed!")

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 
Passed!


In [4]:
def nsquared(n):
    while True:
        yield n ** 2
        n = n - 1
        if n == 0:
            return  # correct way to terminate a generator

In [5]:
gen2 = nsquared(10)

for i in gen2:
    print(i, end=', ')

try:
    next(gen2) # should raise a StopIteration exception
except StopIteration:
    print("\nWe hit the the end of the generator, no more elements can be consumed")
except Exception as e:
    print("\nOops! Unexpected error", e)
finally:
    print("Passed !")

100, 81, 64, 49, 36, 25, 16, 9, 4, 1, 
We hit the the end of the generator, no more elements can be consumed
Passed !


In [6]:
squared_sum1 = sum([i**2 for i in range(1000001)])
print(squared_sum1)

333333833333500000


In [7]:
gen3 = nsquared(1000000)
squared_sum2 = sum(gen3)
print(squared_sum2)

assert squared_sum1 == squared_sum1, "Sums are not equal !"
print("Passed !")

333333833333500000
Passed !


In [8]:
gen4 = nsquared(10)
print(gen4)
gen5 = (i**2 for i in range(11))
print(gen5)

<generator object nsquared at 0x7fef5cb41c00>
<generator object <genexpr> at 0x7fefb94ff030>


In [9]:
# note that the generator has to be reinitialized once it has been consumed
gen4 = nsquared(10)
print(tuple(gen4))
gen4 = nsquared(10)
print(list(gen4))
gen4 = nsquared(10)
print(set(gen4))

print(tuple(i**2 for i in range(11)))
print(list(i**2 for i in range(11)))
print(set(i**2 for i in range(11)))

(100, 81, 64, 49, 36, 25, 16, 9, 4, 1)
[100, 81, 64, 49, 36, 25, 16, 9, 4, 1]
{64, 1, 100, 36, 4, 9, 16, 81, 49, 25}
(0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100)
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
{0, 1, 64, 4, 36, 100, 9, 16, 49, 81, 25}


In [10]:
import numpy as np
print(list(i**2 for i in range(11) if i <=5))
print(list(i**2 if i <=5 else 1 for i in range(11)))
mat = list(i**2 + j**2 if i < j else i + j for i in range(3) for j in range(3))
print(np.array(mat).reshape(3,3))

[0, 1, 4, 9, 16, 25]
[0, 1, 4, 9, 16, 25, 1, 1, 1, 1, 1]
[[0 1 4]
 [1 2 5]
 [2 3 4]]


In [11]:
# Same function, redefined here for clarity
def fib(n):
    a, b = 0, 1
    count = 0
    while 1:
        yield b
        count += 1
        if count == n:
            return
        a, b = b, a + b

def geom(n):
    a = 1
    count = 0
    while True:
        yield a
        count += 1
        if count == n:
            return
        a = a * 2

def constant(n):
    count = 0
    while True:
        yield -1
        count += 1
        if count == n:
            return

In [12]:
def master_sequence(n):
    g1 = fib(n)
    g2 = geom(n)
    g3 = constant(n)
    count = 0
    
    yield from g1
    yield from g2
    yield from g3

In [13]:
master_gen = master_sequence(5) # creates a sequence of length 15
print(list(master_gen))

[1, 1, 2, 3, 5, 1, 2, 4, 8, 16, -1, -1, -1, -1, -1]
